# CSCI 3155: Spring 2023
# Recitation Week 8

### Functions, Closures and Let Rec

## Closures

A *closure* is a way to represent functions in our environments, containing the variable name, expression to be evaluated, and environment to execute the function in.

Why should a closure include an environment?

Consider the example:

~~~
let x = 5 in 
   let addx = function(y) x + y in 
      let x = 10 in 
          addx(10)
~~~

Or equivalently in scala:

~~~
val x = 5;
{
    val addx: Int => Int = (y: Int) => { x+y }
    {
        val x = 10;
        {
            addx(2)
        }
    }
}
~~~

What should this evaluate to?


In [1]:
// try out scala code after discussion
val x = 5;
{
    val addx: Int => Int = (y: Int) => { x+y }
    {
        val x = 10;
        {
            addx(2)
        }
    }
}

x: Int = 5
res0_1: Int = 7

#### NOTE: 
Most languages use static scoping, we want to do the same in Lettuce.

## Exercise 1: Functions with Multiple Parameters

Here, we will explore extending our closures to support multi parameter functions.

For this exercise, we will use a small part of the grammar we have been working with and change our function calls and definitions.

Reference: https://github.com/sriram0339/csci3155_notebooks/blob/master/7/RecursionInLettuce.ipynb

$$
\begin{array}{c|c}
    \text{original grammar} & \text{updated multiparameter function grammar} \\\hline
    \begin{array}{rcl}
    \mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
    \mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
     & | & Ident(\mathbf{Identifier}) \\
     & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) \\ 
     & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
    \end{array}
    &
    \begin{array}{rcll}
    \mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
    \mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
     & | & Ident(\mathbf{Identifier}) \\
     & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & FunDef(\color{red} {\mathbf{Identifier}^*}, \mathbf{Expr}) & \text{Note multiple parameters now possible} \\ 
     & | & FunCall(\mathbf{Expr}, \color{red} {\mathbf{Expr}^*}) & \text{function call - expr(expr1, ... , exprn)} \\
     & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
    \end{array}
\end{array}
$$

Write the scala definition for `FunDef` and `FunCall`. Please use lists to implement the Kleene Star.

#### Note: 

Kleene star, represented by the '$^*$', is a way of representing multiple instances of a something.
It could be one, two, ten, or even none of the item in question.
For example, $\mathbf{Number}^*$ could be no numbers, a single number like $10$, or several numbers, like $10\ 21\ 32$. In Scala, we use `List` datatype to implement Kleene star mechanics.

In [2]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr

//BEGIN SOLUTION
case class FunDef(idList: List[String], e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: List[Expr]) extends Expr
//END SOLUTION

// Let Rec: Revisit after Exercise 1
// BEGIN SOLUTION
case class LetRec(funName: String, paramList: List[String], funExpr: Expr, bodyExpr: Expr) extends Expr
// END SOLUTION 

case class TopLevel(e: Expr) extends Program

defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Let
defined class IfThenElse
defined class Geq
defined class FunDef
defined class FunCall
defined class LetRec
defined class TopLevel

### Environment

We have defined an environment as a function from identifiers to values denoted by them. What sort of environments have we encountered?

* The empty environment: implemented as an empty map. Let us call this environment: $EmptyEnv$
* The environment $\sigma[x \mapsto v]$ which denotes a previously existing environment $\sigma$ extended with the mapping $x \mapsto v$ that associates identifier $x$ with value $v$. Let us call this operation $\texttt{Extend}(\sigma, x, v)$.

#### Revisit this after Exercise 1
* Let us add a third kind of extension to support recursion:
    $\texttt{ExtendRec}(f, x, \texttt{e}, \sigma)$ which creates a new environment $\color{red}{\hat{\sigma}}$ such that $\color{red}{\hat{\sigma}}(x) = \sigma(x)$ for all identifiers $x \not= f$ and $\color{red}{\hat{\sigma}}(f) = \texttt{Closure}(x, \texttt{<body of function expr>}, \color{red}{\hat{\sigma}})$.

### Closure

We will now redefine closures for functions with zero or more args. When our function had one argument, our closures were defined as `Closure(id, expr, env)`. We would now like to define closures as
`Closure( [id1, ..., idn], expr, env)` where 
  - `id1..., idn` are the list of arguments for the function to be called. 
  - `expr` is the body of the function and 
  - `env` is the stored environment for static scoping.
 
 $$\begin{array}{rcl}
 \mathbf{Value} & \Rightarrow & Num(\mathbf{Double}) \\
 & \Rightarrow & Closure(\mathbf{String}^*, \mathbf{Expr}, \mathbf{Environment}) \\
 & \Rightarrow & Error \\
 \end{array}$$

In [3]:
sealed trait Environment 
sealed trait Value

case class Num(d: Double) extends Value
case class Bool(b: Boolean) extends Value
case object Error extends Value
//BEGIN SOLUTION
case class Closure(s: List[String], e: Expr, env: Environment ) extends Value
//END SOLUTION

case object EmptyEnv extends Environment // Represent an environment with nothing in it.
case class Extend(x: String, v: Value, sigma: Environment) extends Environment
//ExtendRec helps me create the illusion of a circular scope

// Note: Revisit this once done with non-recursive multi parameter functions
// Begin Solution
case class ExtendRec(f: String, paramList: List[String], e: Expr, sigma: Environment ) extends Environment
// End Solution

defined trait Environment
defined trait Value
defined class Num
defined class Bool
defined object Error
defined class Closure
defined object EmptyEnv
defined class Extend
defined class ExtendRec

Build an interpreter using the following semantic rules. Ensure that your interpreter correctly deals with the cases that give rise to error by throwing an exception.

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\\end{array} (\text{#3})} $$
$$\newcommand\eval{\mathbf{eval}}$$
$$\semRule{}{\eval(\texttt{FunDef([id1,..., idk], e)},\sigma) = \text{Closure}(\texttt{[id1,..., idk]}, \texttt{e}, \sigma)}{fundef}$$

$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n = k},\ (\forall\ i \in \{ 1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = v_i,  v_i \not= \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \eval(\texttt{fBody}, \color{red}{\sigma_{cl} \circ [id1 \rightarrow v_1, \ldots, idk \rightarrow v_k]})}{funcall-ok}$$
           
Let us interpret this rule.
   - __Purpose__ : Evaluate an expression of the form `FunCall(e, [e1,...,ek])` where `e` is the expr for the called function, and `e1, ..., ek` are exprs for the arguments of this call. There are $k$ arguments.
     - The called function `e` must evaluate to a closure of the form $\text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \sigma_{cl})$.
     - The function call must have the same number of arguments as the closure ($n=k$).
     - Each of the $k$ arguments `ei` must evaluate to a value $v_i$ where $v_i$ is not error.
     - Then the result of evaluating the function call is the same as that of evaluating `fBody` under the environment $\sigma_{cl}$ extended by mapping the formal parameters of the closure, `id1,.., idk`, to $v_1, \ldots, v_k$, respectively.


We can also write some error rules:

$$\semRule {\eval(\texttt{e}, \sigma) \not\in \text{Closure}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-not-a-function}$$


$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n \not= k}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-wrong-num-args}$$
           
  
$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n = k},\ (\exists\ i \in \{1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-arg-error}$$
           
           
**Hint:** Can map, zip, and foldLeft be used for this?
 

In [4]:
class ErrorException(s:String) extends Exception(s){}  

def lookupEnv(sigma: Environment, x: String): Value = sigma match {
    case EmptyEnv => throw new ErrorException(s"Error could not find string $x in environment")
    case Extend(y, v, rest) => {
        if (y == x ) { v}
        else { lookupEnv (rest, x)}
    } 
    // The illusion of circularity is here.
    case ExtendRec(funName, param, funBody, rest) => 
      if (x == funName) 
         Closure(param, funBody, sigma)
      else
         lookupEnv(rest, x)
}


def valueToNumber(v: Value): Double = v match {
    case Num(d) => d
    case _ => throw new ErrorException(s"Could not convert value $v to a number")
}

def eval(e: Expr, env: Environment): Value = {
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(eval(e1, env))
        val v2 = valueToNumber(eval(e2, env))
        val v3 = fun(v1, v2)
        Num(v3)
    }  
    
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(eval(e1, env))
        val v2 = valueToNumber(eval(e2, env))
        val v3 = fun(v1, v2)
        Bool(v3)
    }
    
    e match {
        case Const(d) => Num(d)
        case Ident(x) => lookupEnv(env, x)
        case Plus(e1,e2) => applyArith2(e1, e2)(_ + _)
        case Minus(e1, e2) =>  applyArith2(e1, e2)(_ - _)
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
        
        case Let(id, e1, e2) => {
            val v1 = eval(e1, env)
            val env2 =  Extend(id, v1, env)
            eval(e2, env2)
        }
        
        case IfThenElse(e1, e2, e3) => {
            eval(e1, env) match {
                case Bool(true) => eval(e2, env)
                case Bool(false) => eval(e3, env)
                case _ => throw new ErrorException("Not a bool for Geq operation")
            }
        }
        
        case FunDef(idList, e) => {
            Closure(idList, e, env)
        }
        
        /* the single parameter version of FunCall
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            // Since evaluating e2 did not fail with an exception,
            // if I reach this point in my execution, I know that
            // neither v1 nor v2 are error.
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        // */
        
        case FunCall(e, eList) => {
            //BEGIN SOLUTION
            val v = eval(e, env)
            v match {
                case Closure(argList, fBody, closureEnv) => {
                    if (argList.length == eList.length){
                        val valList = eList.map {eval(_, env)}
                        val zipList = argList.zip(valList)
                        val newMap = zipList.foldLeft[Environment] (env) { (newEnv, newBind) => Extend(newBind._1, newBind._2, newEnv) }
                        eval(fBody, newMap)
                    } else {
                        throw new ErrorException(s"Argument length mismatch")
                    }
                }
                case _ => throw new ErrorException(s"Tried to use a numerical value as a function")
            }
            //END SOLUTION
        }
        
        // Space for LetRec: Revisit later when done Exercise 1
        // BEGIN SOLUTION
        case LetRec(rfun, xList, fExpr, bExpr) => {
            // Storing all the ingredients that are needed to create
            // a circular scope.
            val env2 = ExtendRec(rfun, xList, fExpr, env)
            eval(bExpr, env2)
        }
        // END SOLUTION
    }
}


def evalProgram(p: Program): Value = p match {
    case TopLevel(e) => try
            eval(e, EmptyEnv)
    catch {
        case e: ErrorException => {
            println(e)
            Error
        }
        case e: IllegalArgumentException => {
            println(e)
            Error
        }
        case e => {
            println("Unknown Exception " + e.toString)
            Error
        }
            
    }
}

defined class ErrorException
defined function lookupEnv
defined function valueToNumber
defined function eval
defined function evalProgram

### Tests

In [5]:
//BEGIN TEST
// TEST three arguments
val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val foo = Ident("foo2")
// function(x, y,z)  x + y+z+z
val fun2 = FunDef(List("x", "y", "z"), Plus(Plus(Plus(x, y), z), z))
//let foo = function(x,y,z) x+y+z+z in foo(10, 20, 30)
val l2 = Let("foo2", fun2, FunCall(foo, List(Const(10.0), Const(20.0), Const(30.0))))
//Program
val p2 = TopLevel(l2)
val v2 = evalProgram(p2)
println(s"Your program evaluated to $v2")
assert(v2 == Num(90.0), "Test 2 Failed")
//END TEST

Your program evaluated to Num(90.0)


x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("z")
foo: Ident = Ident("foo2")
fun2: FunDef = FunDef(
  List("x", "y", "z"),
  Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("z")), Ident("z"))
)
l2: Let = Let(
  "foo2",
  FunDef(
    List("x", "y", "z"),
    Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("z")), Ident("z"))
  ),
  FunCall(Ident("foo2"), List(Const(10.0), Const(20.0), Const(30.0)))
)
p2: TopLevel = TopLevel(
  Let(
    "foo2",
    FunDef(
      List("x", "y", "z"),
      Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("z")), Ident("z"))
    ),
    FunCall(Ident("foo2"), List(Const(10.0), Const(20.0), Const(30.0)))
  )
)
v2: Value = Num(90.0)

In [6]:
//BEGIN TEST
//Test zero arguments
val x = Ident("x")
val foo = Ident("foo")
val fcall = FunCall(foo, List())
val fdef = FunDef(List(), x)
val l2 = Let("foo", fdef, fcall)
val l3 = Let("x", Const(10), l2)
val p2 = TopLevel(l3)
val v3 = evalProgram(p2)
println(s"Your program evaluated to $v3")
assert(v3 == Num(10.0), "Test 3 Failed")
//END TEST

Your program evaluated to Num(10.0)


x: Ident = Ident("x")
foo: Ident = Ident("foo")
fcall: FunCall = FunCall(Ident("foo"), List())
fdef: FunDef = FunDef(List(), Ident("x"))
l2: Let = Let("foo", FunDef(List(), Ident("x")), FunCall(Ident("foo"), List()))
l3: Let = Let(
  "x",
  Const(10.0),
  Let("foo", FunDef(List(), Ident("x")), FunCall(Ident("foo"), List()))
)
p2: TopLevel = TopLevel(
  Let(
    "x",
    Const(10.0),
    Let("foo", FunDef(List(), Ident("x")), FunCall(Ident("foo"), List()))
  )
)
v3: Value = Num(10.0)

In [7]:
//BEGIN TEST
//Evaluate with wrong number of args
val x = Ident("x")
val foo = Ident("foo")
val fcall = FunCall(foo, List(Const(1.0)))
val fdef = FunDef(List(), x)
val l2 = Let("foo", fdef, fcall)
val l3 = Let("x", Const(10), l2)
val p4 = TopLevel(l3)
val v4 = evalProgram(p4)
assert(v4 == Error, "Test 4 failed -- your program should have detected that arguments were mismatched")
//END TEST

ammonite.$sess.cmd3$Helper$ErrorException: Argument length mismatch


x: Ident = Ident("x")
foo: Ident = Ident("foo")
fcall: FunCall = FunCall(Ident("foo"), List(Const(1.0)))
fdef: FunDef = FunDef(List(), Ident("x"))
l2: Let = Let(
  "foo",
  FunDef(List(), Ident("x")),
  FunCall(Ident("foo"), List(Const(1.0)))
)
l3: Let = Let(
  "x",
  Const(10.0),
  Let(
    "foo",
    FunDef(List(), Ident("x")),
    FunCall(Ident("foo"), List(Const(1.0)))
  )
)
p4: TopLevel = TopLevel(
  Let(
    "x",
    Const(10.0),
    Let(
      "foo",
      FunDef(List(), Ident("x")),
      FunCall(Ident("foo"), List(Const(1.0)))
    )
  )
)
v4: Value = Error

In [8]:
//BEGIN TEST
// TEST scoping
val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val foo = Ident("foo")

// function(x, y)  x + y + x + z 
val fun = FunDef(List("x", "y"), Plus(Plus(Plus(x, y), x),z))
val funcall = FunCall(foo, List(Const(10.0), Const(30.0)))
// let x = 1 in
//     let z = 50 in
//       let foo = fun(x,y) x + y + x + z in
//         let = 2 in
//           f(10,30)
val l4 = Let("x", Const(1.0), Let("z", Const(50.0), Let("foo", fun, Let("y", Const(2.0), funcall))))
//Program
val p2 = TopLevel(l4)
val v2 = evalProgram(p2)
println(s"Your program evaluated to $v2")
assert(v2 == Num(100.0), "Test 5 Failed == you did not evaluate static scoping correctly")
//END TEST

Your program evaluated to Num(100.0)


x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("z")
foo: Ident = Ident("foo")
fun: FunDef = FunDef(
  List("x", "y"),
  Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("x")), Ident("z"))
)
funcall: FunCall = FunCall(Ident("foo"), List(Const(10.0), Const(30.0)))
l4: Let = Let(
  "x",
  Const(1.0),
  Let(
    "z",
    Const(50.0),
    Let(
      "foo",
      FunDef(
        List("x", "y"),
        Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("x")), Ident("z"))
      ),
      Let(
        "y",
        Const(2.0),
        FunCall(Ident("foo"), List(Const(10.0), Const(30.0)))
      )
    )
  )
)
p2: TopLevel = TopLevel(
  Let(
    "x",
    Const(1.0),
    Let(
      "z",
      Const(50.0),
      Let(
        "foo",
        FunDef(
          List("x", "y"),
          Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("x")), Ident("z"))
        ),
        Let(
          "y",
          Const(2.0),
          FunCall(Ident("foo"), List(Const(10.0), Const(30.0)))
        )
      )
    )
 

## Exercise 2: Recursive multi-param functions

Consider the below recursive function which takes multiple parameters: 

~~~
let rec crazy_rec = function (x, y)
                    if (x >= 0) then 0.5 + x + y
                    else 20 + crazy_rec (0 - x, y)
in 
crazy_rec( -5.0, 2.0)
~~~

We note here that, our grammar so far does not support $let rec$ as of now. Therefore, let us expand the above grammar with let recursion for functions with multiple parameters. 

$$
\begin{array}{c|c}
    \text{original grammar} & \text{updated multiparameter function grammar} \\\hline
    \begin{array}{rcl}
    \mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
    \mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
     & | & Ident(\mathbf{Identifier}) \\
     & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) \\ 
     & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
     & | & \color{red}{LetRec(\mathbf{Identifier},\mathbf{Identifier} ,\mathbf{Expr}, \mathbf{Expr})}
    \end{array}
    &
    \begin{array}{rcll}
    \mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
    \mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
     & | & Ident(\mathbf{Identifier}) \\
     & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & FunDef( \mathbf{Identifier}^*, \mathbf{Expr}) & \text{Note multiple parameters now possible} \\ 
     & | & FunCall(\mathbf{Expr}, \mathbf{Expr}^*) & \text{function call - expr(expr1, ... , exprn)} \\
     & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
     & | & \color{red}{LetRec(\mathbf{Identifier},\mathbf{Identifier^*} ,\mathbf{Expr}, \mathbf{Expr})}
    \end{array}
\end{array}
$$

Now, we have to go back and modify some portions in $eval$ method and define the necessary case classes for making $LetRec$ with multiple parameters happen. 

### Workspace
Space to add your logic here so that grammar reference becomes easy. Once done copy paste your implementation to above.

// Define LetRec
~~~
case class LetRec(funName: String, paramList: List[String], funExpr: Expr, bodyExpr: Expr) extends Expr
~~~

// Define Extend Rec
~~~
case class ExtendRec(f: String, paramList: List[String], e: Expr, sigma: Environment ) extends Environment
~~~

// Space for Letrec eval logic
~~~
case LetRec(rfun, xList, fExpr, bExpr) => {
    // Storing all the ingredients that are needed to create
    // a circular scope.
    val env2 = ExtendRec(rfun, xList, fExpr, env)
    eval(bExpr, env2)
}
~~~

#### Test

Run this test after you are done with making changes as per the new grammar above. 

In [9]:
val crec = Ident("crazy_rec")
val x = Ident("x")
val y = Ident("y")

val e1 = FunCall(crec, List(Const(-5.0), Const(2.0)))
val e21 = Plus(Const(0.5), Plus(x, y))
val e22 = Plus(Const(20.0), FunCall(crec, List(Minus(Const(0), x), y)))
val e2 = LetRec("crazy_rec", List("x", "y"), IfThenElse(Geq(x, Const(0)), e21, e22), e1)

val crazyProg = TopLevel(e2)
assert (evalProgram(crazyProg) == Num(27.5))

crec: Ident = Ident("crazy_rec")
x: Ident = Ident("x")
y: Ident = Ident("y")
e1: FunCall = FunCall(Ident("crazy_rec"), List(Const(-5.0), Const(2.0)))
e21: Plus = Plus(Const(0.5), Plus(Ident("x"), Ident("y")))
e22: Plus = Plus(
  Const(20.0),
  FunCall(Ident("crazy_rec"), List(Minus(Const(0.0), Ident("x")), Ident("y")))
)
e2: LetRec = LetRec(
  "crazy_rec",
  List("x", "y"),
  IfThenElse(
    Geq(Ident("x"), Const(0.0)),
    Plus(Const(0.5), Plus(Ident("x"), Ident("y"))),
    Plus(
      Const(20.0),
      FunCall(
        Ident("crazy_rec"),
        List(Minus(Const(0.0), Ident("x")), Ident("y"))
      )
    )
  ),
  FunCall(Ident("crazy_rec"), List(Const(-5.0), Const(2.0)))
)
crazyProg: TopLevel = TopLevel(
  LetRec(
    "crazy_rec",
    List("x", "y"),
    IfThenElse(
      Geq(Ident("x"), Const(0.0)),
      Plus(Const(0.5), Plus(Ident("x"), Ident("y"))),
      Plus(
        Const(20.0),
        FunCall(
          Ident("crazy_rec"),
          List(Minus(Const(0.0), Ident("x")), Iden

## That's all folks !